# Test Data Checker and Exporter

## I. Setup Code

In [14]:
import sys
sys.path.append('/home/frc1318/scouting/scouting2022/scouter')

In [15]:
import pickle
import sqlite3
import pathlib
import json

import pandas as pd
import xlsxwriter

import server.event as event
import server.database as database
import server.tba as tba
import server.config as config

In [16]:
con = database.get_con()

## II. Code to Set Event and Current Match

In [ ]:
event.set_current_event("2022testdata")

In [ ]:
event.set_current_match("qm4")

## III. Code to Review Scouting Data

In [17]:
def get_match_measures(match):
    qry = """
        SELECT * FROM Measures
         WHERE match = ?
      ORDER BY team_number;"""
    return pd.read_sql(qry, con, params=(match,))

pd.set_option('display.max_rows', None)

def match_teams(match):
    qry = f"""SELECT Matches.alliance, Matches.station, Matches.team_number, Teams.team_name
               FROM Matches
               LEFT JOIN Teams
               ON Matches.team_number = Teams.team_number
               WHERE Matches.match='{match}';"""
    teams_df = pd.read_sql(qry, con)
    return teams_df

def get_team_measures(team):
    qry = f"SELECT * FROM Measures WHERE team_number = '{team}'"
    team_df = pd.read_sql(qry, con)
    return team_df

def spot_check_measures(match = None, team_number = None, alliance = None, station = None):
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    query = """
    SELECT * FROM Measures
    WHERE match = ?
    AND team_number = ?
    ORDER BY task, phase;
    """
    if (team_number is None and alliance is not None and station is not None):
        team_number = event.get_team(match, alliance, station, con=con)
    cur.execute(query, (match, team_number))
    rows = cur.fetchall()
    measures = [dict(row) for row in rows]
    con.row_factory = None
    return pd.DataFrame(measures)

## IV. Code to Write Data to File

In [ ]:
matches = pd.read_sql("SELECT * FROM Matches;", con)
print(matches.shape)
matches.head()

In [ ]:
measures = pd.read_sql("SELECT * FROM Measures;", con)
print(measures.shape)
measures.head()

In [ ]:
status = pd.read_sql("SELECT * FROM Status;", con)
status

In [ ]:
teams = pd.read_sql("SELECT * FROM Teams;", con)
print(teams.shape)
teams.head()

In [ ]:
with open("2022testdata.pickle", "wb") as pfile:
    pickle.dump({"matches": matches, "measures": measures, "status": status, "teams": teams}, pfile)

## Write Data to Excel File

In [ ]:
#writes measures, teams, matches, and status tables to different sheets in dfs.xlsx file in the notebooks folder
excel_path = pathlib.Path().absolute().joinpath('dfs.xlsx')
with pd.ExcelWriter(excel_path) as writer:
    measures.to_excel(writer, sheet_name = 'measures')
    teams.to_excel(writer, sheet_name = 'teams')
    matches.to_excel(writer, sheet_name  = 'matches')
    status.to_excel(writer, sheet_name = 'status')

## Widgets

In [18]:
from ipywidgets import widgets
from IPython.display import display
import IPython

In [19]:
def print_df(button=None):
    with out:
        if funcw.value == 'spot_check':
            display(spot_check_measures(match = matchp.value, team_number = teamp.value))
        elif funcw.value == 'match_measures':
            match_df = get_match_measures(matchp.value)
#             display(match_df)
            display(match_df[match_df['measure1']!='0'])
        elif funcw.value == 'team_measures':
            display(get_team_measures(teamp.value))
        elif funcw.value == 'match_teams':
            display(match_teams(matchp.value))
def clear_print(button=None):
    out.clear_output()

team_df = pd.read_sql('SELECT team_number AS team FROM Teams', con)
team_list = team_df['team'].tolist()
match_df = pd.read_sql('SELECT DISTINCT match FROM Matches;', con)
match_list = match_df[[x.startswith('qm') for x in match_df['match']]]['match'].tolist()
match_list.sort(key = lambda x: int(x[2:]))

funcw = widgets.ToggleButtons(
    options=['spot_check','match_measures', 'team_measures','match_teams'],
    description='measures',
    disabled=False)
matchp = widgets.Dropdown(
    options = match_list,
    value='qm1',
    description='match:',
    disabled=False)
teamp = widgets.Dropdown(
    options = team_list,
    value=None,
    description='team:',
    disabled=False)
show_button = widgets.Button(description = 'show data')
clear_button = widgets.Button(description = 'clear output')
out = widgets.Output(layout={'border': '1px solid black'})
def show_dfs2(button=None):
    with out2:
        if funce.value == 'year_events':
            events_df = pd.DataFrame(tba.get_events_year(year_text.value))
            display(events_df[['end_date', 'key','city','state_prov']])
        if funce.value == 'year_districts':
            districts_df = pd.DataFrame(tba.get_districts(year_text.value))
            display(districts_df)
        if funce.value == 'district_events':
            try:
                events_df = pd.DataFrame(tba.get_events(year_text.value))
                display(events_df[['state_prov','city', 'end_date', 'key']])
            except:
                print('Invalid district code')
def clear2(button=None):
    out2.clear_output()
funce = widgets.ToggleButtons(
    options=['year_events', 'year_districts', 'district_events'],
    description='events',
    disabled=False)

year_text = widgets.Text(
    value='2022',
    description='Year/district:',
    disabled=False)
show_button2 = widgets.Button(description = 'show data')
clear_button2 = widgets.Button(description = 'clear output')
out2 = widgets.Output(layout={'border': '1px solid black'})

In [20]:
print("""Arguments needed for each measure:
spot_check: match, team
match_measures: match
team_meausres: team
match_teams: match""")
display(funcw)
display(teamp)
display(matchp)
display(show_button)
show_button.on_click(print_df)
display(clear_button)
clear_button.on_click(clear_print)
display(out)

Arguments needed for each measure:
spot_check: match, team
match_measures: match
team_meausres: team
match_teams: match


ToggleButtons(description='measures', options=('spot_check', 'match_measures', 'team_measures', 'match_teams')…

Dropdown(description='team:', options=('frc1294', 'frc1318', 'frc1425', 'frc1778', 'frc1899', 'frc2046', 'frc2…

Dropdown(description='match:', options=('qm1', 'qm2', 'qm3', 'qm4', 'qm5', 'qm6', 'qm7', 'qm8', 'qm9', 'qm10',…

Button(description='show data', style=ButtonStyle())

Button(description='clear output', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [ ]:
display(funce)
display(year_text)
display(show_button2)
show_button2.on_click(show_dfs2)
display(clear_button2)
clear_button2.on_click(clear2)
display(out2)

In [16]:
event.import_measures('331test.csv')

In [11]:
pd.read_sql('SELECT * FROM Measures WHERE team_number = "frc1318" AND phase = "tele" AND task = "upper";', con)

,match,team_number,phase,task,measure1,measure2,measure_type
0,qm1,frc1318,tele,upper,14,4,count
1,qm9,frc1318,tele,upper,74,6,count
2,qm15,frc1318,tele,upper,4,4,count


In [12]:
event.export_measures('331test.csv')

In [13]:
pd.read_sql('SELECT * FROM Measures;', con).tail()

,match,team_number,phase,task,measure1,measure2,measure_type
3714,qm32,frc4682,auto,taxi,false,-1,boolean
3715,qm32,frc4682,endgame,temp_disabled,false,-1,boolean
3716,qm32,frc4682,auto,upper,0,0,count
3717,qm32,frc4682,tele,upper,0,0,count
3718,qm32,frc2412,auto,start_pos,right,-1,categorical


In [ ]:
"""INSERT INTO Teams (team_number, team_name, city, state, country)
        VALUES ('frc360', 'The Revolution', 'Issaquah', 'Washington', 'USA');"""

In [13]:
con.close()